<a href="https://colab.research.google.com/github/TimOgden/RoastBot/blob/master/RoastTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [58]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, num_heads, fully_connected_units=256, dropout=.5, name=None):
    super().__init__(name=name)
    self.self_attention = tf.keras.layers.MultiHeadAttention(num_heads, key_dim=768)
    self.fc1 = tf.keras.layers.Dense(fully_connected_units, activation=tf.keras.activations.gelu)
    self.fc2 = tf.keras.layers.Dense(fully_connected_units, activation=tf.keras.activations.gelu)
    self.dropout = tf.keras.layers.Dropout(dropout)
    self.layer_norm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()
  def __call__(self, x, training=True):
    attention_weights, attention_scores = self.self_attention(x, x, return_attention_scores=True)
    normalized = self.layer_norm(self.add([x, attention_weights]))
    x = self.fc2(self.dropout(self.fc1(normalized)))
    return self.layer_norm(self.add([x, normalized])), attention_scores

In [59]:
encoder_layer = EncoderLayer(8, name='test_layer')

In [60]:
input_tensor = tf.keras.Input(shape=[None, 16*16])

In [62]:
output_tensor, output_weights = encoder_layer(input_tensor, input_tensor)
print(output_tensor.shape, output_weights.shape)

(None, None, 256) (None, 8, None, None)
